# Simple EDA

In [2]:
import pandas as pd
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import count
from snowflake.snowpark.context import get_active_session

/home/roman/Dev/Python/snowflake_project/.venv/lib/python3.12/site-packages/numpy/_core/getlimits.py:552: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)
/home/roman/Dev/Python/snowflake_project/.venv/lib/python3.12/site-packages/snowflake/snowpark/session.py:38: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
session = get_active_session()
df = session.table("RAW.CUSTOMER_CHURN_RAW")

# Basic info (logged, but not returned)
print("📊 Row count:", df.count())
df.print_schema()

print("\n🔍 Sample data:")
df.show(10)

# Categorical value counts
print("\n🧑‍🤝‍🧑 Gender distribution:")
df.group_by("GENDER").agg(count("*").alias("count")).show()

print("\n🌍 Geography distribution:")
df.group_by("GEOGRAPHY").agg(count("*").alias("count")).show()

print("\n💳 Card types:")
df.group_by("CARD_TYPE").agg(count("*").alias("count")).show()

# Binary flags
print("\n💳 Has credit card:")
df.group_by("HASCRCARD").agg(count("*").alias("count")).show()

print("\n🔥 Active members:")
df.group_by("ISACTIVEMEMBER").agg(count("*").alias("count")).show()

# Target variable
print("\n🚪 Exited distribution:")
df.group_by("EXITED").agg(count("*").alias("count")).show()

# Numerical summary
print("\n📈 Summary statistics (AGE, BALANCE, CREDITSCORE):")
df.select("AGE", "BALANCE", "CREDITSCORE").describe().show()

print("\n💰 Salary vs. Satisfaction:")
df.select("ESTIMATEDSALARY", "SATISFACTION_SCORE").describe().show()


SnowparkSessionException: (1403): No default Session is found. Please create a session before you call function 'udf' or use decorator '@udf'.

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import count
from dotenv import load_dotenv
import os
import toml
from pathlib import Path

load_dotenv()
snowflake_password = os.getenv("PASSWORD")

# Get connections.toml path
current_notebook_dir = Path(os.getcwd())
config_file_path = current_notebook_dir.parent / 'connections.toml'

try:
    with open(config_file_path, 'r') as f:
        config = toml.load(f)
    
    if "connections" not in config or "my_connection" not in config["connections"]:
        raise ValueError(f"Nie znaleziono sekcji 'my_example_connection' w pliku '{config_file_path}'.")
    
    connection_parameters = config["connections"]["my_example_connection"]

except FileNotFoundError:
    raise FileNotFoundError(f"Plik konfiguracyjny '{config_file_path}' nie został znaleziony. Upewnij się, że ścieżka jest poprawna.")
except toml.TomlDecodeError as e:
    raise ValueError(f"Błąd parsowania pliku TOML '{config_file_path}': {e}. Sprawdź składnię pliku.")
except Exception as e:
    raise Exception(f"Wystąpił nieoczekiwany błąd podczas wczytywania konfiguracji: {e}")

connection_parameters['password'] = snowflake_password
session = Session.builder.configs(connection_parameters).create()

print("Connected to Snowflake!")
print(f"Database: {session.get_current_database()}")
print(f"Schema: {session.get_current_schema()}")
print(f"Warehouse: {session.get_current_warehouse()}")

# Get data
df = session.table("RAW.CUSTOMER_CHURN_RAW")


print("\n📊 Row count:", df.count())
df.print_schema()

print("\n🔍 Sample data:")
df.show(10)

print("\n🧑‍🤝_ Gender distribution:")
df.group_by("GENDER").agg(count("*").alias("count")).show()

print("\n🌍 Geography distribution:")
df.group_by("GEOGRAPHY").agg(count("*").alias("count")).show()

print("\n💳 Card types:")
df.group_by("CARD_TYPE").agg(count("*").alias("count")).show()

print("\n💳 Has credit card:")
df.group_by("HASCRCARD").agg(count("*").alias("count")).show()

print("\n🔥 Active members:")
df.group_by("ISACTIVEMEMBER").agg(count("*").alias("count")).show()

print("\n🚪 Exited distribution:")
df.group_by("EXITED").agg(count("*").alias("count")).show()

print("\n📈 Summary statistics (AGE, BALANCE, CREDITSCORE):")
df.select("AGE", "BALANCE", "CREDITSCORE").describe().show()

print("\n💰 Salary vs. Satisfaction:")
df.select("ESTIMATEDSALARY", "SATISFACTION_SCORE").describe().show()

# 7. Zamknij sesję
session.close()
print("\nSession closed!")

Połączono z Snowflake!
Baza danych: "CHURN_PROJECT"
Schemat: None
Warehouse: "ROMAN_WH"

📊 Row count: 10000
root
 |-- "ROWNUMBER": LongType() (nullable = True)
 |-- "CUSTOMERID": StringType(16777216) (nullable = True)
 |-- "SURNAME": StringType(16777216) (nullable = True)
 |-- "CREDITSCORE": LongType() (nullable = True)
 |-- "GEOGRAPHY": StringType(16777216) (nullable = True)
 |-- "GENDER": StringType(16777216) (nullable = True)
 |-- "AGE": LongType() (nullable = True)
 |-- "TENURE": LongType() (nullable = True)
 |-- "BALANCE": DoubleType() (nullable = True)
 |-- "NUMOFPRODUCTS": LongType() (nullable = True)
 |-- "HASCRCARD": LongType() (nullable = True)
 |-- "ISACTIVEMEMBER": LongType() (nullable = True)
 |-- "ESTIMATEDSALARY": DoubleType() (nullable = True)
 |-- "EXITED": LongType() (nullable = True)
 |-- "COMPLAIN": LongType() (nullable = True)
 |-- "SATISFACTION_SCORE": LongType() (nullable = True)
 |-- "CARD_TYPE": StringType(16777216) (nullable = True)
 |-- "POINT_EARNED": LongTy